## Some examples for Working with geotiffs

Using IOOS3 conda environment

A good source for info is http://www.gis.usu.edu/~chrisg/python/2009/

In [1]:
from osgeo import gdal, gdalnumeric, ogr, osr
import sys
gdal.UseExceptions()

fn = "D:/crs/proj/2015_Sandwich/Sandwich_all_surveys_preliminary/2016-03-30_DEM_10cm_cropped.tif"
ds=gdal.Open(fn)
rows = ds.RasterYSize
cols = ds.RasterXSize
bands = ds.RasterCount
print("bands: ",bands, "rows: ", rows, "cols: ",cols)

# get georeference info
transform = ds.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = transform[5]
print("xOrigin, yOrigin: ",xOrigin,yOrigin)

# read an n-dimensional array into an ndArray Appropriate when RasterCount > 1.
nda=ds.ReadAsArray()
print("Shape of array: ",nda.shape)

# different way of accessing transformation info
# coordinates are located at the top left corner of the pixels
ulx, xres, xskew, uly, yskew, yres  = ds.GetGeoTransform()
lrx = ulx + (ds.RasterXSize * xres)
lry = uly + (ds.RasterYSize * yres)
print("upper left x, y: ", ulx,uly)
print("lower right x, y: ", lrx,lry)

bands:  1 rows:  12265 cols:  13052
xOrigin, yOrigin:  376206.5224272934 4625437.967243103
Shape of array:  (12265, 13052)
upper left x, y:  376206.5224272934 4625437.967243103
lower right x, y:  377511.72242729337 4624211.467243103


In [2]:
# get the pixel coordinates for some point x, y
x = 377000.
y = 4624000.
xOffset = int((x-xOrigin) / pixelWidth)
yOffset = int((y-yOrigin) / pixelHeight)
print("pixx, pixy: ",xOffset,yOffset)

pixx, pixy:  7934 14379


#### Get the projection and print in nice format

In [3]:
raster_wkt = ds.GetProjection()
spatial_ref = osr.SpatialReference()
spatial_ref.ImportFromWkt(raster_wkt)
print(spatial_ref.ExportToPrettyWkt())

PROJCS["NAD83 / UTM zone 19N",
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.2572221010042,
                AUTHORITY["EPSG","7019"]],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433],
        AUTHORITY["EPSG","4269"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-69],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AUTHORITY["EPSG","26919"]]


#### or in Proj4 format

In [4]:
print(spatial_ref.ExportToProj4())

+proj=utm +zone=19 +datum=NAD83 +units=m +no_defs 


In [5]:
# Load the source data as a gdalnumeric array
srcArray = gdalnumeric.LoadFile(fn)

# Also load as a gdal image to get geotransform
# (world file) info
srcImage = gdal.Open(fn)
geoTrans = srcImage.GetGeoTransform()
print(geoTrans)


(376206.5224272934, 0.09999999999999822, 0.0, 4625437.967243103, 0.0, -0.10000000000003037)


### Get some info on a band number

In [6]:
def ds_info( band_num, input_file ):
    src_ds = gdal.Open( input_file )
    if src_ds is None:
        print('Unable to open {%s}'.format(input_file))

    try:
        srcband = src_ds.GetRasterBand(band_num)
    except RuntimeError as e:
        print('No band {%i} found'.format(band_num))
        print(e)

    print("[ NO DATA VALUE ] = {}".format(srcband.GetNoDataValue()))
    print("[ MIN ] = {}".format(srcband.GetMinimum()))
    print("[ MAX ] = {}".format(srcband.GetMaximum()))
    print ("[ SCALE ] = {}".format(srcband.GetScale()))
    print ("[ UNIT TYPE ] = {}".format(srcband.GetUnitType()))
    ctable = srcband.GetColorTable()

    if ctable is None:
        print('No ColorTable found')

In [7]:
ds_info(1,fn)

[ NO DATA VALUE ] = -32767.0
[ MIN ] = None
[ MAX ] = None
[ SCALE ] = 1.0
[ UNIT TYPE ] = metre
No ColorTable found


In [9]:
# Here is an example of a command line that
# gdalbuildvrt -separate -te xmin ymin xmax ymax -input_file_list my_filenames.txt output_file.vrt

In [10]:
import subprocess
p=subprocess.run(['gdalinfo',fn], stdout=subprocess.PIPE)
print(p)

CompletedProcess(args=['gdalinfo', 'D:/crs/proj/2015_Sandwich/Sandwich_all_surveys_preliminary/2016-03-30_DEM_10cm_cropped.tif'], returncode=0, stdout=b'Driver: GTiff/GeoTIFF\r\nFiles: D:/crs/proj/2015_Sandwich/Sandwich_all_surveys_preliminary/2016-03-30_DEM_10cm_cropped.tif\r\nSize is 13052, 12265\r\nCoordinate System is:\r\nPROJCS["NAD83 / UTM zone 19N",\r\n    GEOGCS["NAD83",\r\n        DATUM["North_American_Datum_1983",\r\n            SPHEROID["GRS 1980",6378137,298.2572221010042,\r\n                AUTHORITY["EPSG","7019"]],\r\n            AUTHORITY["EPSG","6269"]],\r\n        PRIMEM["Greenwich",0],\r\n        UNIT["degree",0.0174532925199433],\r\n        AUTHORITY["EPSG","4269"]],\r\n    PROJECTION["Transverse_Mercator"],\r\n    PARAMETER["latitude_of_origin",0],\r\n    PARAMETER["central_meridian",-69],\r\n    PARAMETER["scale_factor",0.9996],\r\n    PARAMETER["false_easting",500000],\r\n    PARAMETER["false_northing",0],\r\n    UNIT["metre",1,\r\n        AUTHORITY["EPSG","9001"